In [5]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

class TrainBookingSystem:
    def __init__(self, booked_seats_data):
        self.total_rows = 10
        self.total_cols = 4
        self.seats = [['O' for _ in range(self.total_cols)] for _ in range(self.total_rows)]
        self.booked_seats = {}
        self.base_seat_price = 800  

        for index, row in booked_seats_data.iterrows():
            if row['Seat Booked (Y/N)'] == 'Y':
                seat_number = row['Seat Number']
                age_group = row['Preferred Age Group']
                self.booked_seats[seat_number] = age_group
                self.seats[row['Row']][row['Column']] = 'X'

        self.model = self.train_model(booked_seats_data)

        self.root = tk.Tk()
        self.root.title("Train Booking System")
        self.age_group = None
        self.recommendations = []

        self.create_age_selection()

    def train_model(self, booked_seats_data):
        booked_seats_data['Age Group Encoded'] = pd.Categorical(booked_seats_data['Preferred Age Group']).codes

        X = booked_seats_data[['Row', 'Column', 'Age Group Encoded']]
        y = booked_seats_data['Seat Booked (Y/N)'].apply(lambda x: 1 if x == 'Y' else 0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        return model

    def create_age_selection(self):
        self.age_selection_frame = tk.Frame(self.root)
        self.age_selection_frame.pack(pady=20)

        tk.Label(self.age_selection_frame, text="Select Age Group:").pack(side=tk.LEFT)
        self.age_group_var = tk.StringVar(value='18-25')
        age_groups = ['18-25', '25-40', '40-60', '60+']
        for age_group in age_groups:
            tk.Radiobutton(self.age_selection_frame, text=age_group, variable=self.age_group_var, value=age_group).pack(side=tk.LEFT)

        tk.Button(self.age_selection_frame, text="Confirm Age Group", command=self.confirm_age_group).pack(side=tk.LEFT)

    def confirm_age_group(self):
        self.age_group = self.age_group_var.get()
        self.recommendations = self.get_recommended_seats(self.age_group)
        self.show_recommendations()

    def show_recommendations(self):
        self.age_selection_frame.pack_forget()
        self.recommendation_frame = tk.Frame(self.root)
        self.recommendation_frame.pack(pady=20)

        tk.Label(self.recommendation_frame, text=f"Recommended Seats for {self.age_group}:").pack()

        recommended_seats_str = ', '.join(map(str, self.recommendations))
        tk.Label(self.recommendation_frame, text=f"Seats: {recommended_seats_str}").pack()
        tk.Button(self.recommendation_frame, text="Proceed to Seat Selection", command=self.create_seat_grid).pack(pady=10)

    def create_seat_grid(self):
        self.recommendation_frame.pack_forget()
        self.grid_frame = tk.Frame(self.root)
        self.grid_frame.pack(pady=20)

        self.buttons = []
        for r in range(self.total_rows):
            row_buttons = []
            for c in range(self.total_cols):
                seat_number = r * self.total_cols + c + 1
                if seat_number in self.booked_seats:
                    color = 'red'  
                elif seat_number in self.recommendations:
                    color = 'blue'  
                else:
                    color = 'green'  
                
                button = tk.Button(self.grid_frame, text=seat_number, width=5, height=2, bg=color,
                                   command=lambda sn=seat_number: self.select_seat(sn))
                button.grid(row=r, column=c)
                row_buttons.append(button)
            self.buttons.append(row_buttons)

    def get_recommended_seats(self, age_group):
        recommended = []

        age_groups = ['18-25', '25-40', '40-60', '60+']
        age_group_label = pd.Categorical([age_group], categories=age_groups).codes[0]

        features_list = []

        for r in range(self.total_rows):
            for c in range(self.total_cols):
                seat_number = r * self.total_cols + c + 1

                if seat_number not in self.booked_seats:
                    if self.is_adjacent_seat_available(r, c, age_group):

                        features_list.append({
                            'Row': r,
                            'Column': c,
                            'Age Group Encoded': age_group_label
                        })

        features = pd.DataFrame(features_list)
        if not features.empty:
            predictions = self.model.predict(features)
            for i, row in features.iterrows():
                seat_number = row['Row'] * self.total_cols + row['Column'] + 1
                if predictions[i] >= 0.5:  
                    recommended.append(seat_number)

        return recommended

    def is_adjacent_seat_available(self, row, col, age_group):
        adjacent_seats = [
            (row, col - 1), (row, col + 1)  
        ]
        for r, c in adjacent_seats:
            if 0 <= r < self.total_rows and 0 <= c < self.total_cols:
                seat_number = r * self.total_cols + c + 1
                if seat_number in self.booked_seats and self.booked_seats[seat_number] == age_group:
                    return True
        return False

    def select_seat(self, seat_number):
        row = (seat_number - 1) // self.total_cols
        col = (seat_number - 1) % self.total_cols

        if seat_number in self.booked_seats:
            messagebox.showwarning("Seat Already Booked", f"Seat {seat_number} is already booked.")
        else:
            self.book_seat(seat_number)

    def get_dynamic_price(self):
        remaining_seats_count = (self.total_rows * self.total_cols) - len(self.booked_seats)
        total_seats = self.total_rows * self.total_cols

        if remaining_seats_count > total_seats * 0.5:
            multiplier = 1.0  
        elif remaining_seats_count > total_seats * 0.2:
            multiplier = 1.2  
        else:
            multiplier = 1.5  

        return self.base_seat_price * multiplier

    def book_seat(self, seat_number):
        row = (seat_number - 1) // self.total_cols
        col = (seat_number - 1) % self.total_cols
        price = self.get_dynamic_price()

        self.booked_seats[seat_number] = self.age_group
        self.seats[row][col] = 'X'
        
        self.buttons[row][col].config(bg='red')
        
        messagebox.showinfo("Booking.......", f"Seat {seat_number} booking is in progress at Price: ₹{price:.2f}")

        self.buttons[row][col].config(state=tk.DISABLED)

        self.reset_to_age_selection()

    def reset_to_age_selection(self):
        self.grid_frame.pack_forget()
        self.create_age_selection()

    def cancel_booking(self, seat_number):
        if seat_number in self.booked_seats:
            refund_amount = self.base_seat_price * 0.6
            del self.booked_seats[seat_number]
            row = (seat_number - 1) // self.total_cols
            col = (seat_number - 1) % self.total_cols
            self.seats[row][col] = 'O'
            
            self.buttons[row][col].config(bg='green')
            messagebox.showinfo("Cancellation", f"Booking for seat {seat_number} cancelled. Refund: ₹{refund_amount:.2f}")
        else:
            messagebox.showwarning("Warning", "No booking found for that seat number.")

    def run(self):
        self.root.mainloop()

# Load your dataset
booking_data = pd.read_csv('train_booking_data.csv')  
booking_system = TrainBookingSystem(booking_data)
booking_system.run()
